<a href="https://colab.research.google.com/github/thee-smarty/Camouflage-Military-Soldier-Density-Estimation/blob/main/Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 13.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
from PIL import Image
import numpy as np
import cv2
from ultralytics import YOLO

# Load the model (ensure the path is correct)
model_path = '/content/sample_data/model.pt'
model = YOLO(model_path)

# Define a function to count the number of bounding boxes
def count_boxes(results):
    """Count the number of bounding boxes in the detection results."""
    try:
        # Ensure the results object has the expected structure
        if hasattr(results[0], 'boxes'):
            return len(results[0].boxes)
        else:
            return 0
    except Exception as e:
        print(f"Error in counting boxes: {str(e)}")
        return 0

# Modify the predict_image function to include box counting
def predict_image(input_image):
    try:
        # Convert the input image to the format needed by YOLO
        image_pil = input_image.convert("RGB")
        image_array = np.array(image_pil)

        # Run prediction
        results = model.predict(source=image_array, imgsz=640, conf=0.5)

        # Count the number of bounding boxes
        num_boxes = count_boxes(results)

        # Annotate the image with bounding boxes and display the number of boxes
        annotated_image = results[0].plot(line_width=2)  # Thicker box lines for visibility

        # Convert the annotated image from BGR to RGB
        annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)

        # Optionally, add the number of boxes as text on the image
        # cv2.putText(annotated_image_rgb, f"People Detected: {num_boxes}", (10, 30),
        #             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Return the annotated image as a PIL image
        return f"Soldiers Detected: {num_boxes}",Image.fromarray(annotated_image_rgb)

    except Exception as e:
        # Print error message for debugging
        print(f"Error during prediction : {str(e)}")
        return None

# Create the Gradio interface (for v3.x+)
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Input is PIL image
    outputs=[gr.Textbox(label="Estimation of Camouflage Military"),gr.Image(type="pil")],  # Output is PIL image
    live=True,
    title="Camouflaged Military Soldier Density Estimation",
    description="Upload an image to detect and count military soldiers",
)

# Launch the interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e52be0d9586fefdea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
